## 축제 기간 Stay 데이터 분석

#### 원본 데이터 병합 및 행정동 변환

In [ ]:
import os
import pandas as pd

def merge_csv(folder):
    csv_files = []
    for file_name in os.listdir(folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder, file_name)
            df = pd.read_csv(file_path)
            csv_files.append(df)
    merged = pd.concat(csv_files, ignore_index=True)
    return merged

In [ ]:
stay1 = merge_csv('stay_20230901_15')
stay2 = merge_csv('stay_20230916_30')
stay3 = merge_csv('stay_20231001_15')

In [ ]:
#행정동 변환하기
hdong = pd.read_csv('데이터분석 분야_데이터정의서\KIKmix_20230701.csv')

hdong = hdong.groupby(['행정동코드','시도명','시군구명','읍면동명'], as_index=False).first()
hdong['주소'] = (hdong['시도명'] + ' ' + hdong['시군구명'] + ' ' + hdong['읍면동명']).fillna('').str.strip()

In [ ]:
def change_hdong(file):
    file = file.merge(hdong[['행정동코드','주소']], how='left', left_on='hdong_cd', right_on='행정동코드')
    file['hdong_cd'] = file['주소'].fillna(file['hdong_cd'])
    file.drop(columns=['행정동코드','주소'], inplace=True)

    return file

In [ ]:
stay_sep1 = change_hdong(stay1)
stay_sep2 = change_hdong(stay2)
stay_oct1 = change_hdong(stay3)

### 0. 데이터 필터링
| 축제 이름             | 비축제 기간         | 축제 기간       |분석 지역          |
|----------------------|--------------------|----------------|-------------------|
| 소래포구 축제         | 09/01 - 09/14     | 09/15 - 09/17 | 축제 개최 행정동 (논현1동,논현2동)|
| 진주남강유등축제      | 09/23 - 10/07     | 10/08 - 10/15 | 축제지 소재 법정동 (경상남도 진주시 성북동, 천전동, 신안동)|
- 외지인은 출발지가 축제 개최지 (시 기준) 외의 지역인 사람으로 정의

In [22]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
stay_sep1 = pd.read_csv('stay_sep1.csv')
stay_sep2 = pd.read_csv('stay_sep2.csv')
stay_oct1 = pd.read_csv('stay_oct1.csv')

In [8]:
# 진주 남강 유등축제 데이터 추출
parks = ['경상남도 진주시 성북동', '경상남도 진주시 천전동', '경상남도 진주시 신안동']
jinju = stay_oct1[stay_oct1['hdong_cd'].isin(parks)] 

# 인천 소래포구 축제 데이터 추출
ipark = ['인천광역시 남동구 논현1동', '인천광역시 남동구 논현2동']
prev1 = stay_sep1[(stay_sep1['date']>=20230907) & (stay_sep1['hdong_cd'].isin(ipark))]
prev2 = stay_sep2[(stay_sep2['date']<=20230917) & (stay_sep2['hdong_cd'].isin(ipark))]

incheon = pd.concat([prev1, prev2])

In [13]:
# 축제 기간 / 비축제 기간 구분
incheon['축제'] = '소래포구 축제'
jinju['축제'] = '진주남강유등 축제'

incheon['기간'] = incheon['date'].apply(lambda x : '축제' if (x >= 20230915) & (x <= 20230917) else '비축제')
jinju['기간'] = jinju['date'].apply(lambda x : '축제' if (x >= 20231008) & (x <= 20231015) else '비축제')

### 축제/비축제 기간 일별 여행-쇼핑-여가 목적 평균 체류 인원 
`분석 목표` : 축제 기간과 비축제기간 사이 여행/쇼핑/여가 목적 체류 인원 증가 추이 파악

#### 데이터 추가 필터링 및 연산

In [91]:
# 요일별 평균 체류 인원 수
incheon_trip =incheon[(incheon['purpose']==3) | (incheon['purpose']==5)] 
jinju_trip =jinju[(jinju['purpose']==3) | (jinju['purpose']==5)]

df_weekend = pd.concat([incheon_trip, jinju_trip])

df_weekend['date'] = pd.to_datetime(df_weekend['date'],format='%Y%m%d') 
df_weekend['weekday'] = df_weekend['date'].dt.day_name()

df_sep = df_weekend.groupby(['축제', 'date','기간','weekday'])['stay_cnts'].mean().reset_index() 

df_sep.head()

,축제,date,기간,weekday,stay_cnts
0,소래포구 축제,2023-09-07,비축제,Thursday,84.394505
1,소래포구 축제,2023-09-08,비축제,Friday,91.292632
2,소래포구 축제,2023-09-09,비축제,Saturday,144.063444
3,소래포구 축제,2023-09-10,비축제,Sunday,128.785714
4,소래포구 축제,2023-09-11,비축제,Monday,80.914286


#### 시각화

In [78]:
fig = go.Figure()

incheon_trip = df_sep[df_sep['축제']=='소래포구 축제']
incheon_trip['date'] = incheon_trip['date'].astype(str)

fig = px.line(incheon_trip, x='date', y='stay_cnts', color_discrete_sequence=['#0e6d62'],
              labels={'date': '날짜', 'stay_cnts': '일별 체류 인원'},
              title='소래포구 축제 기간의 일별 평균 체류 인원')

# X축에서 모든 '축제' 기간에 해당하는 날짜가 표시되도록 설정 (categoryorder 사용)
fig.update_xaxes(type='category', categoryorder='array', categoryarray=incheon_trip['date'])

# 그래프 출력
fig.show()

In [84]:
fig = go.Figure()

jinju_trip = df_sep[df_sep['축제']=='진주남강유등 축제']
jinju_trip['date'] = jinju_trip['date'].astype(str)

fig = px.line(jinju_trip, x='date', y='stay_cnts', color_discrete_sequence=['#0e6d62'],
              labels={'date': '날짜', 'stay_cnts': '일별 평균 체류 인원'},
              title='진주남강유등 축제 기간의 일별 평균 체류 인원')

# X축에서 모든 '축제' 기간에 해당하는 날짜가 표시되도록 설정 (categoryorder 사용)
fig.update_xaxes(type='category', categoryorder='array', categoryarray=incheon_trip['date'])

# 그래프 출력
fig.show()

- 축제기간 / 비축제기간 요일별 평균 체류 인원 수

In [88]:
# 요일 순서를 정의
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 요일 순서를 카테고리로 변환
df_sep['weekday'] = pd.Categorical(df_sep['weekday'], categories=weekday_order, ordered=True)

# 요일별 평균 체류 인원 계산
df_sep = df_sep.groupby(['축제', '기간', 'weekday'])['stay_cnts'].mean().reset_index() 

# 진주남강유등 축제의 축제와 비축제 기간 데이터 필터링
jinju_fest = df_sep[(df_sep['축제'] == '진주남강유등 축제') & (df_sep['기간'] == '축제')]
jinju_notfest = df_sep[(df_sep['축제'] == '진주남강유등 축제') & (df_sep['기간'] == '비축제')]

# Plotly 그래프 생성
fig = go.Figure()

# 축제 기간 데이터 추가
fig.add_trace(go.Scatter(x=jinju_fest['weekday'], y=jinju_fest['stay_cnts'], mode='lines+markers',
                         name='축제 기간', line=dict(color='blue', dash='solid')))

# 비축제 기간 데이터 추가
fig.add_trace(go.Scatter(x=jinju_notfest['weekday'], y=jinju_notfest['stay_cnts'], mode='lines+markers',
                         name='비축제 기간', line=dict(color='green', dash='dash')))

# 레이아웃 설정
fig.update_layout(
    title='진주남강유등 축제 : 요일별 평균 쇼핑/여행/여가 목적 체류 인원 변화 (축제 기간 vs 비축제 기간)',
    xaxis_title='요일',
    yaxis_title='평균 체류 인원',
    legend_title='기간',
    xaxis=dict(categoryorder='array', categoryarray=weekday_order)  # x축에 요일 순서 지정
)

# 그래프 출력
fig.show()

In [89]:
# 요일 순서를 정의
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 요일 순서를 카테고리로 변환
df_sep['weekday'] = pd.Categorical(df_sep['weekday'], categories=weekday_order, ordered=True)

# 요일별 평균 체류 인원 계산
df_sep = df_sep.groupby(['축제', '기간', 'weekday'])['stay_cnts'].mean().reset_index() 

# 축제와 비축제 기간 데이터 필터링
fest = df_sep[(df_sep['축제'] == '소래포구 축제') & (df_sep['기간'] == '축제')]
notfest = df_sep[(df_sep['축제'] == '소래포구 축제') & (df_sep['기간'] == '비축제')]

# Plotly 그래프 생성
fig = go.Figure()

# 축제 기간 데이터 추가
fig.add_trace(go.Scatter(x=fest['weekday'], y=fest['stay_cnts'], mode='lines+markers',
                         name='축제 기간', line=dict(color='blue', dash='solid')))

# 비축제 기간 데이터 추가
fig.add_trace(go.Scatter(x=notfest['weekday'], y=notfest['stay_cnts'], mode='lines+markers',
                         name='비축제 기간', line=dict(color='green', dash='dash')))

# 레이아웃 설정
fig.update_layout(
    title='소래포구 축제 : 요일별 평균 쇼핑/여행/여가 목적 체류 인원 변화 (축제 기간 vs 비축제 기간)',
    xaxis_title='요일',
    yaxis_title='평균 체류 인원',
    legend_title='기간',
    xaxis=dict(categoryorder='array', categoryarray=weekday_order)  # x축에 요일 순서 지정
)

# 그래프 출력
fig.show()

### 시간별 평균 체류 인원 
`분석 목표` : 축제기간 시간대별 평균 체류 인원 수 파악 (쇼핑/여행/여가 목적 방문객에 한정)

#### 추가 전처리 및 연산

In [97]:
incheon_trip =incheon[(incheon['purpose']==3) | (incheon['purpose']==5)] 
jinju_trip =jinju[(jinju['purpose']==3) | (jinju['purpose']==5)]

incheon_fest = incheon_trip[(incheon_trip['축제'] == '소래포구 축제') & (incheon_trip['기간'] == '축제')]
jinju_fest = jinju_trip[(jinju_trip['축제'] == '진주남강유등 축제') & (jinju_trip['기간'] == '축제')]

fest = pd.concat([incheon_fest, jinju_fest])

fest = fest.groupby(['축제', 'time'])['stay_cnts'].mean().reset_index()
fest.head()

,축제,time,stay_cnts
0,소래포구 축제,08:00,109.360000
1,소래포구 축제,09:00,71.740331
2,소래포구 축제,10:00,88.707071
3,소래포구 축제,11:00,112.465686
4,소래포구 축제,12:00,140.029557


#### 시각화

In [98]:
fig = go.Figure()

incheon_fest = fest[fest['축제']=='소래포구 축제']
jinju_fest = fest[fest['축제']=='진주남강유등 축제']

# incheon 데이터 선 추가
fig.add_trace(go.Scatter(x=incheon_fest['time'], y=incheon_fest['stay_cnts'], mode='lines', 
                         name='Incheon', line=dict(color='#0e6d62')))

# jinju 데이터 선 추가
fig.add_trace(go.Scatter(x=jinju_fest['time'], y=jinju_fest['stay_cnts'], mode='lines', 
                         name='Jinju', line=dict(color='red')))

# 레이아웃 설정
fig.update_layout(
    title='축제 기간 시간대별 평균 체류 인원수',
    xaxis_title='시간대',
    yaxis_title='평균 체류 인원수'
)

# 그래프 출력
fig.show()